In [5]:
import sys
import random
import operator
import pandas as pd
import numpy as np
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

In [3]:
def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}q\n'.format(i, feat))
    outfile.close()

In [4]:
def runXGB(train_X, train_y, test_X, test_y = None, test_X2 = None, feature_names = None, seed_val = 0, rounds = 50, dep = 8, eta = 0.05):
    params = {}
    params["objectives"] = "binary:logistic"
    params['eval_metric'] = 'auc'
    params['eta'] = eta
    params['subsample'] = 0.7
    params['min_child_weight'] = 1
    params['colsample_bytree'] = 0.7
    params["max_depth"] = dep

    params["silent"] = 1
    params["seed"] = seed_val

    num_rounds = rounds

    plst = list(params.items())
    xgtrain = xgb.DMatrix(train_X, label = train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label = test_y)
        watchlist = [ (xgtrain, 'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=100, verbose_eval=20)
    else:
		xgtest = xgb.DMatrix(test_X)
		model = xgb.train(plst, xgtrain, num_rounds)

	if feature_names is not None:
		create_feature_map(feature_names)
		model.dump_model('xgbmodel.txt', 'xgb.fmap', with_stats=True)
		importance = model.get_fscore(fmap='xgb.fmap')
		importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
		imp_df = pd.DataFrame(importance, columns=['feature','fscore'])
		imp_df['fscore'] = imp_df['fscore'] / imp_df['fscore'].sum()
		imp_df.to_csv("imp_feat.txt", index=False)

	pred_test_y = model.predict(xgtest, ntree_limit=model.best_ntree_limit)
	pred_test_y2 = model.predict(xgb.DMatrix(test_X2), ntree_limit=model.best_ntree_limit)

	loss = 0
	if test_y is not None:
		loss = metrics.roc_auc_score(test_y, pred_test_y)
		return pred_test_y, loss, pred_test_y2
	else:
		return pred_test_y, loss, pred_test_y2

IndentationError: expected an indented block (Temp/ipykernel_11868/920627856.py, line 1)